### Install dependencies
Install the python dependencies required for our application. Using a Python virtual environment is usually a good idea.

In [ ]:
# Code cell 1

%pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken redis langchain

### Import libraries and set up Azure OpenAI and Redis connection info
Fill in your Azure OpenAI and Azure Cache for Redis information below. This will be used later to establish the connection these services, generate the embeddings, and load them into Redis. This example stores these values in application variables for the sake of simplicity. Outside of tutorials, it's strongly recommended to store these in environment variables or using a secrets manager like Azure KeyVault. 

Note that there are differences  between the `OpenAI` and `Azure OpenAI` endpoints. This example uses the configuration for `Azure OpenAI`. See [How to switch between OpenAI and Azure OpenAI endpoints with Python](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/switching-endpoints) for more details. 

In [2]:
# Code cell 2

import re
from num2words import num2words
import os
import pandas as pd
import tiktoken
from typing import List
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores.redis import Redis as RedisVectorStore
from langchain.document_loaders import DataFrameLoader


API_KEY = "<azure-openai-key>"
RESOURCE_ENDPOINT = "<your-azure-openai-endpoint>" # e.g. https://openaiexample.openai.azure.com/
DEPLOYMENT_NAME = "<name-of-your-model-deployment>" # this is the name you selected for your deployment, not the name of the OpenAI model. 
MODEL_NAME = "text-embedding-ada-002" # this is the name of the OpenAI model. 
REDIS_ENDPOINT = "<your-azure-redis-endpoint>" # must include port at the end. e.g. "redisdemo.eastus.redisenterprise.cache.azure.net:10000"
REDIS_PASSWORD = "<your-azure-redis-password>"


### Import dataset

This example uses the [Wikipedia Movie Plots](https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots) dataset from Kaggle. Download this file and place it in the same directory as this jupyter notebook.  

In [3]:
# Code cell 3

df=pd.read_csv(os.path.join(os.getcwd(),'wiki_movie_plots_deduped.csv'))
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


Process the dataset to remove spaces in the column titles and filter the dataset to lower the size. This isn't required, but is helpful in reducing the time it takes to generate embeddings and loading the index into Redis. Feel free to play around with the filters, or add your own! 

In [4]:
# Code cell 4

df.insert(0, 'id', range(0, len(df)))
df['year'] = df['Release Year'].astype(int)
df['origin'] = df['Origin/Ethnicity'].astype(str)
del df['Release Year']
del df['Origin/Ethnicity']
df = df[df.year > 1970] # only movies made after 1970
df = df[df.origin.isin(['American','British','Canadian'])] # only movies from English-speaking cinema
df

,id,Title,Director,Cast,Genre,Wiki Page,Plot,year,origin
8626,8626,$ aka Dollars,Richard Brooks,"Warren Beatty, Goldie Hawn",unknown,https://en.wikipedia.org/wiki/$_(film),"Set in Hamburg, West Germany, several criminal...",1971,American
8627,8627,200 Motels,"Tony Palmer, Charles Swenson","Frank Zappa, Ringo Starr, Theodore Bikel",unknown,https://en.wikipedia.org/wiki/200_Motels,"In 200 Motels, the film attempts to portray th...",1971,American
8628,8628,The Anderson Tapes,Sidney Lumet,"Sean Connery, Dyan Cannon, Christopher Walken,...",unknown,https://en.wikipedia.org/wiki/The_Anderson_Tapes,"Burglar John ""Duke"" Anderson (Sean Connery) is...",1971,American
8629,8629,The Andromeda Strain,Robert Wise,"Arthur Hill, James Olson, Kate Reid, David Way...",unknown,https://en.wikipedia.org/wiki/The_Andromeda_St...,"After a satellite, a U.S. government project c...",1971,American
8630,8630,Bad Man's River,Eugenio Martin,"Lee Van Cleef, Gina Lollobrigida",unknown,https://en.wikipedia.org/wiki/Bad_Man%27s_River,Roy King's gang robs a bank and flees to Mexic...,1971,American
...,...,...,...,...,...,...,...,...,...
22428,22428,"Hochelaga, Land of Souls (Hochelaga terre des ...",François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...",2017,Canadian
22429,22429,Indian Horse,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...",2017,Canadian
22430,22430,The Little Girl Who Was Too Fond of Matches (L...,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",2017,Canadian
22431,22431,Meditation Park,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",2017,Canadian


Remove whitespace from the `Plot` column to make it easier to generate embeddings.

In [5]:
# Code cell 5

pd.options.mode.chained_assignment = None

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['Plot']= df['Plot'].apply(lambda x : normalize_text(x))

Calculate the number of tokens required to generate the embeddings for this dataset. You may want to filter the dataset more stringently in order to limit the tokens required. 

In [6]:
# Code cell 6

tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["Plot"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
print('Number of movies: ' + str(len(df))) # print number of movies remaining in dataset
print('Number of tokens required:' + str(df['n_tokens'].sum())) # print number of tokens

Number of movies: 11125
Number of tokens required:7044844


### Load Dataframe into LangChain
Using the `DataFrameLoader` class allows you to load a pandas dataframe into LangChain. That makes it easy to load your data and use it to generate embeddings using LangChain's other integrations.

In [7]:
# Code cell 7

loader = DataFrameLoader(df, page_content_column="Plot" )
movie_list = loader.load()

### Generate embeddings and Load them into Azure Cache for Redis
Using LangChain, this example connects to Azure OpenAI Service to generate embeddings for the dataset. These embeddings are then loaded into [Azure Cache for Redis](https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/cache-overview), a fully managed Redis service on Azure. Make sure you're using the [Enterprise tier](https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/quickstart-create-redis-enterprise) of Azure Cache for Redis, which features the [RediSearch](https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/cache-redis-modules#redisearch) module that includes vector search capability. Finally, a copy of the index schema is saved. That is useful for loading the index into Redis later if you don't want to regenerate the embeddings.

In [8]:
# Code cell 8

# we will use OpenAI as our embeddings provider
embedding = AzureOpenAIEmbeddings(
    deployment=DEPLOYMENT_NAME,
    model=MODEL_NAME,
    azure_endpoint=RESOURCE_ENDPOINT,
    openai_api_type="azure",
    openai_api_key=API_KEY,
    openai_api_version="2023-05-15",
    show_progress_bar=True,
    chunk_size=16 # current limit with Azure OpenAI service. This will likely increase in the future.
    )

# name of the Redis search index to create
index_name = "movieindex"

# create a connection string for the Redis Vector Store. Uses Redis-py format: https://redis-py.readthedocs.io/en/stable/connections.html#redis.Redis.from_url
# This example assumes TLS is enabled. If not, use "redis://" instead of "rediss://
redis_url = "rediss://:" + REDIS_PASSWORD + "@"+ REDIS_ENDPOINT

# create and load redis with documents
vectorstore = RedisVectorStore.from_documents(
    documents=movie_list,
    embedding=embedding,
    index_name=index_name,
    redis_url=redis_url
)

# save index schema so you can reload in the future without re-generating embeddings
vectorstore.write_schema("redis_schema.yaml")

# This may take up to 10 minutes to complete.

### Run search queries
Using the vectorstore we just built in LangChain, we can conduct similarity searches using the `similarity_search_with_score` method. In this example, the top 10 results for a given query are returned.

In [32]:
# Code cell 9

results = vectorstore.similarity_search_with_score("Spaceships, aliens, and heroes saving America", k=10)

for i, j in enumerate(results):
    movie_title = str(results[i][0].metadata['Title'])
    similarity_score = str(round((1 - results[i][1]),4))
    print(movie_title + ' (Score: ' + similarity_score + ')')


Independence Day (Score: 0.8348)
The Flying Machine (Score: 0.8332)
Remote Control (Score: 0.8301)
Bravestarr: The Legend (Score: 0.83)
Xenogenesis (Score: 0.8291)
Invaders from Mars (Score: 0.8291)
Apocalypse Earth (Score: 0.8287)
Invasion from Inner Earth (Score: 0.8287)
Thru the Moebius Strip (Score: 0.8283)
Solar Crisis (Score: 0.828)


### Run hybrid queries

You can also run hybrid queries. That is, queries that use both vector search and filters based on other parameters in the dataset. In this case, we filter our query results to only movies tagged with the `comedy` genre. One of the advantages of using LangChain with Redis is that metadata is preserved in the index, so you can use it to filter your results. 

In [33]:
# Code cell 10

from langchain.vectorstores.redis import RedisText

genre_filter = RedisText("Genre") == "comedy"
results = vectorstore.similarity_search_with_score("Spaceships, aliens, and heroes saving America", filter=genre_filter, k=10)
for i, j in enumerate(results):
    movie_title = str(results[i][0].metadata['Title'])
    similarity_score = str(round((1 - results[i][1]),4))
    print(movie_title + ' (Score: ' + similarity_score + ')')

Remote Control (Score: 0.8301)
Meet Dave (Score: 0.8236)
Elf-Man (Score: 0.8208)
Fifty/Fifty (Score: 0.8167)
Mars Attacks! (Score: 0.8165)
Strange Invaders (Score: 0.8143)
Amanda and the Alien (Score: 0.8136)
Suburban Commando (Score: 0.8129)
Coneheads (Score: 0.8129)
Morons from Outer Space (Score: 0.8121)


### Appendix A: Load index data already in Redis
If you already have embeddings data in Redis, you can load it into your LangChain vectorstore oboject using the `from_existing_index` method. This is useful if you don't want to re-run your embeddings model. You'll need to provide the index schema that was saved when you generated the embeddings.

In [5]:
# Code cell 11

vectorstore = RedisVectorStore.from_existing_index(
    embedding=embedding,
    redis_url=redis_url,
    index_name=index_name,
    schema="redis_schema.yaml"
)